In [ ]:
from peewee import *
from random import randint
from typing import Any, Callable, TypeVar
import json

In [ ]:
T = TypeVar('T')

In [ ]:
def pad_start(width, fillchar, number) -> int:
    return int(str(number).rjust(width, fillchar))

def pad_end(width, fillchar, number) -> int:
    return int(str(number).ljust(width, fillchar))

In [ ]:

def get_random_integer(min_value: T = 1, max_value: T = 9) -> int:
    min = pad_start(7,'1', pad_end(6, '0', min_value))
    max = pad_start(8, '9', pad_end(7, '9', max_value))
    return randint(min, max)

rand_number = get_random_integer()

print(rand_number)

In [ ]:
database_name = 'notebooks_2_{}.db'.format(get_random_integer())
print(database_name)

In [ ]:
db = SqliteDatabase(database_name)

In [ ]:
dir(db)

In [ ]:
db.connection();

In [ ]:
class BaseModel(Model):
    class Meta:
        database = db

In [ ]:
class Person(BaseModel):
    id = AutoField()
    first_name = CharField()
    last_name = CharField()
    phone = CharField(null=True)
    
class Experience(BaseModel):
    id = AutoField()
    title = CharField()
    company = CharField()
    start_date = DateField()
    end_date = DateField(null=True)
    person = ForeignKeyField(Person, backref='experiences')
    
class Contact(BaseModel):
    id = AutoField()
    nickname = CharField()
    owner = ForeignKeyField(Person, backref='contacts')

class Phone(BaseModel):
    id = AutoField()
    type = CharField()
    number = CharField()
    contact = ForeignKeyField(Contact, backref='phones')


In [ ]:
db.create_tables([Person, Experience, Contact, Phone])

In [ ]:
data = {
    'person_record': {
        'first_name': 'Gabriel',
        'last_name': 'Ariza',
        'phone': '1234567890'
    },
    'experience_records': [
        {
            'title'      :      'Developer',
            'company'    :      'Google',
            'start_date' :      '2018-01-01',
            'end_date'   :      '2018-02-01',
            'person_id'     :      0
        }
    ],
    'contact_records': [
        {
            'nickname': 'john_doe',
            'owner_id': 0
        }
    ],

    'phone_records': [
        {
            'type': 'home',
            'number': '1234567890',
            'contact_id': 0
        }
    ]
}


In [ ]:
insert_data = data.get('person_record', {})
print(json.dumps([insert_data], indent=4))

In [ ]:
insert_response = Person.create(**insert_data)
print(json.dumps([insert_response.__data__], indent=4))
# dir(insert_response)

In [ ]:
insert_data = data.get('person_record', {})
insert_data['experiences'] = data.get('experience_records', [])
insert_data['contacts'] = data.get('contact_records', [])
for i, contact in enumerate(insert_data['contacts']):
    contact.update({'nickname': '{}. {}'.format((i + 1), contact.get('nickname', ''))})
    contact['phones'] = data.get('phone_records', [])

print(json.dumps([insert_data], indent=4))

In [ ]:
insert_response = Person.create(**insert_data)
try:
    insert_response.__data__['experiences'] = insert_response.experiences
    insert_response.__data__['contacts'] = insert_response.contacts
except Exception as e:
    print(e)

print(json.dumps([insert_response.__data__], indent=4))
dir(insert_response)

In [ ]:

dir(insert_response.__data__)

In [ ]:
print(json.dumps([{'{}'.format(i):d} for i, d in insert_response.__data__.items()], indent=2))

In [ ]:
print(json.dumps(insert_response.experiences, indent=4))

In [ ]:
print(json.dumps(insert_response.contacts, indent=4))

In [ ]:
print(insert_response.id)

In [ ]:
experiences = Experience().select()

In [ ]:
print([exp for exp in experiences])

In [ ]:
print([Experience(**exp).save() for exp in insert_response.experiences if not exp.update({'person_id':insert_response.id})])

In [ ]:
print(json.dumps(insert_response.experiences, indent=2))

In [ ]:


def bulk_create_experience(experiences, insert_response):
    for experience in experiences:
        experience.save()
        insert_response.append(experience)
    return insert_response